In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

# %pip install --upgrade pip
%pip install shapely

from shapely.geometry import shape
import json

#df_geo = spark.read.format("delta").load("abfss://6a2d5314-0cf7-4016-a2f6-97ffae7192ad@onelake.dfs.fabric.microsoft.com/30fdebf4-ca3e-422a-8387-5f4d13fe2abf/Tables/gold_fact_ward_stats_toronto")
# df_geo = spark.read.table("gold_fact_ward_stats_toronto")

df_existing = spark.read.table("silver_01_dim_ward_geojson_toronto")

# Convert Spark DataFrame to Pandas for geometry processing
df_geo_pd = df_existing.select("_id", "geometry").toPandas()

def get_centroid(geom_str):
    try:
        geom_json = json.loads(geom_str)  # Convert string to dict
        geom = shape(geom_json)           # Convert dict to shapely geometry
        return geom.centroid.y, geom.centroid.x
    except Exception as e:
        print(f"Error parsing geometry: {e}")
        return None, None

df_geo_pd["centroid_lat"], df_geo_pd["centroid_lon"] = zip(*df_geo_pd["geometry"].apply(get_centroid))

# Convert back to Spark DataFrame
df_geo_centroids = spark.createDataFrame(df_geo_pd)
df_geo_centroids.show(1)

df_updated = df_existing.join(df_geo_centroids.select("_id", "centroid_lat", "centroid_lon"), on="_id", how="left")
df_updated.show(1)

#spark.sql("DROP TABLE IF EXISTS silver_dim_ward_centroid_toronto")
df_updated.write.option("mergeSchema", "true").mode("overwrite").saveAsTable("silver_02_dim_ward_centroid_toronto")

# Final Output as CSV
df = spark.read.table("silver_02_dim_ward_centroid_toronto")
pdf = df.toPandas()
pdf.to_csv("/lakehouse/default/Files/Silver_Exports/silver_02_dim_ward_centroid_toronto.csv", index=False)

display(pdf) # To get a download option as csv


In [5]:
df = spark.read.format("csv").option("header","true").load("Files/Silver_Exports/silver_02_dim_ward_centroid_toronto.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Silver_Exports/silver_02_dim_ward_centroid_toronto.csv".
display(df)

StatementMeta(, 6c214a08-4a1e-4823-882c-615966e783d3, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 771a71db-508a-4294-b68d-a4f9f38bd396)